### Building an image generation app 

In [ ]:
from diffusers import DiffusionPipeline

pipeline = DiffusionPipeline.from_pretrained("runwayml/stable-diffusion-v1-5")

def get_completion(prompt, parameters=None):
    if parameters is not None:
        return pipeline(prompt, **parameters).images[0]
    return pipeline(prompt).images[0]

Using Gradio Interface

In [ ]:
import io
import base64
import gradio as gr

from PIL import Image

def base64_to_pil(img_base64):
    base64_decoded = base64.b64decode(img_base64)
    byte_stream = io.BytesIO(base64_decoded)
    pil_image = Image.open(byte_stream)
    return pil_image

def generate(prompt, negative_prompt, steps, guidance, width, height):
    params = {
        "negative_prompt": negative_prompt,
        "num_inference_steps": steps,
        "guidance_scale": guidance,
        "width": width,
        "height": height
    }

    output = get_completion(prompt, params)
    pil_image = base64_to_pil(output)
    return pil_image

demo = gr.Interface(
    fn=generate,
    inputs=[
        gr.Textbox(label="Your prompt"),
        gr.Textbox(label="Negative prompt"),
        gr.Slider(
            label="Inference Steps", minimum=1, maximum=100, value=25,
            info="In how many steps will the denoiser denoise the image?"
        ),
        gr.Slider(
            label="Guidance Scale", minimum=1, maximum=20, value=7,      
            info="Controls how much the text prompt influences the result"
        ),
        gr.Slider(label="Width", minimum=64, maximum=512, step=64, value=512),
        gr.Slider(label="Height", minimum=64, maximum=512, step=64, value=512),
    ],
    outputs=[gr.Image(label="Result")],
    title="Image Generation with Stable Diffusion",
    description="Generate any image with Stable Diffusion",
    allow_flagging="never"
)

demo.launch()

Using Gradio Blocks

In [ ]:
with gr.Blocks() as demo:
    
    gr.Markdown("# Image Generation with Stable Diffusion")
    
    with gr.Row():
        with gr.Column(scale=4):
            # Give prompt some real estate
            prompt = gr.Textbox(label="Your prompt") 
        with gr.Column(scale=1, min_width=50):
            # Submit button side by side!
            btn = gr.Button("Submit") 
    
    # Let's hide the advanced options!
    with gr.Accordion("Advanced options", open=False): 
        negative_prompt = gr.Textbox(label="Negative prompt")
        with gr.Row():
            with gr.Column():
                steps = gr.Slider(label="Inference Steps", minimum=1, maximum=100, value=25,
                                  info="In many steps will the denoiser denoise the image?")
                guidance = gr.Slider(label="Guidance Scale", minimum=1, maximum=20, value=7,
                                     info="Controls how much the text prompt influences the result")
            with gr.Column():
                width = gr.Slider(label="Width", minimum=64, maximum=512, step=64, value=512)
                height = gr.Slider(label="Height", minimum=64, maximum=512, step=64, value=512)
    
    # Move the output up too
    output = gr.Image(label="Result") 

    btn.click(fn=generate, inputs=[prompt,negative_prompt,steps,guidance,width,height], outputs=[output])

gr.close_all()
demo.launch()